In [141]:
import os
import sys
import pandas as pd
import random
import json
import regex as re

In [12]:
current_dir = os.path.dirname(os.path.realpath("python_coursework.ipynb"))
resource_dir = os.path.join(current_dir,"resources")

In [ ]:
global game_state
game_state = {}



In [111]:
global game_history
game_history = []

In [37]:
[0] *6

[0, 0, 0, 0, 0, 0]

In [143]:
def persist_game_stats(game_state):
    with open("game_stats.json", "w") as file:
        json.dump(game_state, file)

In [ ]:
def get_secret_word(wordle_words_file, word_length):
    secret_word = ''
    file_name = os.path.join(resource_dir,wordle_words_file)
    if os.path.isfile(file_name):
        with open(file_name, 'r') as file:
            lines = [line.strip() for line in file.readlines() if len(line.strip())==word_length]
            if len(lines)==0:
                raise Exception("No valid words in wordle words file")
            else:
                secret_word = random.choice(lines)
    else:
        raise Exception("File is not found Error")
    return secret_word

In [3]:
['']*5

['', '', '', '', '']

In [ ]:
special_char = re.findall(r"[^a-zA-Z0-9]+", 'stare+')
index = "stare+".find(special_char[0])
word_str = "stare+"[:index]


'stare'

In [147]:
re.split(r"[^a-zA-Z0-9]+", 'stare+')

['stare', '']

In [112]:
def initialize_game_history(max_guesses):
    game_hist_len =  int(max_guesses)+1
    game_history_list = [0] * game_hist_len
    with open('game_history.txt','w') as f:
        f.write(",".join(map(str,game_history_list)))
    game_history.append({'game_hist_length':game_hist_len,'game_hist_list':game_history_list,
    'old_game_hist_list':[]})
    print(game_history)

In [121]:
def update_guess_limit_in_hist(game_history,game_state):
    max_guess_limit=game_state['max_guesses']
    game_hist_len = len(game_history)
    max_guess_hist = game_history[game_hist_len-1]['game_hist_length']
    game_history_list = game_history[game_hist_len-1]['game_hist_list'].copy()
    if max_guess_hist-1 != max_guess_limit:
        new_guess_hist_limit = int(max_guess_limit)+1
        new_list_size = new_guess_hist_limit-max_guess_hist
        extendable_list = [0]*new_list_size
        old_game_hist_list = game_history_list.copy()
        game_history_list.extend(extendable_list)
        game_history.append({'game_hist_length':new_guess_hist_limit,'game_hist_list':game_history_list,
                             'old_game_hist_list':old_game_hist_list})
        with open('game_history.txt','w') as f:
            f.write(",".join(map(str,game_history_list)))


In [132]:
def update_game_history(game_history,game_state):
    game_status = game_state['status']
    game_hist_len = len(game_history)
    game_history_list = game_history[game_hist_len-1]['game_hist_list'].copy()
    if game_status == 'lost':
        game_history_list[0]+=1
    elif game_status == 'won':
        guess_count = int(game_state['i'])+1
        game_history_list[guess_count]+=1
    game_history[game_hist_len-1]['game_hist_list'] = game_history_list
    with open('game_history.txt','w') as f:
        f.write(",".join(map(str,game_history_list)))
    print(game_history)

In [114]:
initialize_game_history(6)

[{'game_hist_length': 7, 'game_hist_list': [0, 0, 0, 0, 0, 0, 0], 'old_game_hist_list': []}]


In [124]:
game_state['status']='won'
game_state['i'] = 3

In [126]:

game_history,game_state

([{'game_hist_length': 7,
   'game_hist_list': [0, 0, 0, 0, 0, 0, 0],
   'old_game_hist_list': []},
  {'game_hist_length': 8,
   'game_hist_list': [0, 0, 0, 0, 0, 0, 0, 0],
   'old_game_hist_list': [0, 0, 0, 0, 0, 0, 0]}],
 {'secret_word': 'lunge',
  'word_length': 5,
  'max_guesses': 7,
  'grid': [['', '', '', '', ''],
   ['', '', '', '', ''],
   ['', '', '', '', ''],
   ['', '', '', '', ''],
   ['', '', '', '', ''],
   ['', '', '', '', ''],
   ['', '', '', '', '']],
  'color_grid': [['w', 'w', 'w', 'w', 'w'],
   ['w', 'w', 'w', 'w', 'w'],
   ['w', 'w', 'w', 'w', 'w'],
   ['w', 'w', 'w', 'w', 'w'],
   ['w', 'w', 'w', 'w', 'w'],
   ['w', 'w', 'w', 'w', 'w'],
   ['w', 'w', 'w', 'w', 'w']],
  'i': 3,
  'j': 0,
  'status': 'won',
  'wordle_words_file': 'wordle_words.txt',
  'all_words_file': 'english_dict.txt',
  'popup_queue': [],
  'active_letters': {'A': True,
   'B': True,
   'C': True,
   'D': True,
   'E': True,
   'F': True,
   'G': True,
   'H': True,
   'I': True,
   'J': True,
 

In [133]:
update_game_history(game_history,game_state)

[{'game_hist_length': 7, 'game_hist_list': [0, 0, 0, 0, 0, 0, 0], 'old_game_hist_list': []}, {'game_hist_length': 8, 'game_hist_list': [0, 0, 0, 0, 2, 0, 0, 0], 'old_game_hist_list': [0, 0, 0, 0, 0, 0, 0]}]


In [ ]:
def initialize_game(all_words_file, wordle_words_file=None,secret_word=None, max_guesses=6,word_length=5):
    game_state ={}
    try:
        if os.path.exists("game_state.json"):
            with open("game_state.json", "r") as file:
                game_state = json.load(file)
        else:
            initialize_game_history(max_guesses)
            if wordle_words_file is None:
                wordle_words_file = 'wordle_words.txt'
            if all_words_file is None:
                all_words_file = 'english_dict.txt'
            if secret_word is None:
                secret =  get_secret_word(wordle_words_file, word_length)
            else:
                if len(secret_word)==word_length:
                    secret = secret_word
                else:
                    raise Exception("The secret word is not of the correct length")
            if max_guesses == 0:
                raise Exception("Max guesses must be 1 or more")
            if word_length == 0:
                raise Exception("Word length must be 1 letter or more")
            
            game_state['secret_word'] = secret
            game_state['word_length'] = word_length
            game_state['max_guesses'] = max_guesses

            guess_nested_list = []
            for i in range(max_guesses):
                guess_list = [''] * int(word_length)
                guess_nested_list.append(guess_list)
            game_state['grid'] = guess_nested_list
            color_nested_list = []
            for i in range(max_guesses):
                color_list = ['w'] * int(word_length)
                color_nested_list.append(color_list)
            #print(f"Color List : {color_nested_list}")
            game_state['color_grid'] = color_nested_list
            game_state['i'] = 0
            game_state['j'] = 0
            game_state['status'] = 'playing'
            game_state['wordle_words_file'] = 'wordle_words.txt'
            game_state['all_words_file'] = all_words_file
            game_state['popup_queue'] = []
            game_state['active_letters'] = {
                "A": True, "B": True, "C": True, "D": True, "E": True, "F": True, 
                "G": True, "H": True, "I": True, "J": True, "K": True, "L": True, 
                "M": True, "N": True, "O": True, "P": True, "Q": True, "R": True, 
                "S": True, "T": True, "U": True, "V": True, "W": True, "X": True, 
                "Y": True, "Z": True
            }
            update_guess_limit_in_hist(game_history=game_history,game_state=game_state)
    except KeyboardInterrupt as err:
        persist_game_stats(game_state)
        print(str(err))
    except SystemExit as sysexit:
        persist_game_stats(game_state)
        print(str(sysexit))
    except Exception as e:
        persist_game_stats(game_state)
        print(str(e))
    return game_state




In [25]:
game_state

{'secret_word': 'gravy',
 'word_length': 6,
 'max_guesses': 7,
 'grid': [['', '', '', '', '', ''],
  ['', '', '', '', '', ''],
  ['', '', '', '', '', ''],
  ['', '', '', '', '', ''],
  ['', '', '', '', '', ''],
  ['', '', '', '', '', ''],
  ['', '', '', '', '', '']],
 'color_grid': [['w', 'w', 'w', 'w', 'w', 'w'],
  ['w', 'w', 'w', 'w', 'w', 'w'],
  ['w', 'w', 'w', 'w', 'w', 'w'],
  ['w', 'w', 'w', 'w', 'w', 'w'],
  ['w', 'w', 'w', 'w', 'w', 'w'],
  ['w', 'w', 'w', 'w', 'w', 'w'],
  ['w', 'w', 'w', 'w', 'w', 'w']],
 'i': 0,
 'j': 0,
 'status': 'playing',
 'wordle_words_file': 'wordle_words.txt',
 'all_words_file': 'english_dict.txt',
 'popup_queue': [],
 'active_letters': {'A': True,
  'B': True,
  'C': True,
  'D': True,
  'E': True,
  'F': True,
  'G': True,
  'H': True,
  'I': True,
  'J': True,
  'K': True,
  'L': True,
  'M': True,
  'N': True,
  'O': True,
  'P': True,
  'Q': True,
  'R': True,
  'S': True,
  'T': True,
  'U': True,
  'V': True,
  'W': True,
  'X': True,
  'Y': T

In [119]:
game_state = initialize_game(all_words_file='english_dict.txt',max_guesses=7,word_length=5)
game_state

['cigar', 'rebut', 'sissy', 'humph', 'awake', 'blush', 'focal', 'evade', 'naval', 'serve', 'heath', 'dwarf', 'model', 'karma', 'stink', 'grade', 'quiet', 'bench', 'abate', 'feign', 'major', 'death', 'fresh', 'crust', 'stool', 'colon', 'abase', 'marry', 'react', 'batty', 'pride', 'floss', 'helix', 'croak', 'staff', 'paper', 'unfed', 'whelp', 'trawl', 'outdo', 'adobe', 'crazy', 'sower', 'repay', 'digit', 'crate', 'cluck', 'spike', 'mimic', 'pound', 'maxim', 'linen', 'unmet', 'flesh', 'booby', 'forth', 'first', 'stand', 'belly', 'ivory', 'seedy', 'print', 'yearn', 'drain', 'bribe', 'stout', 'panel', 'crass', 'flume', 'offal', 'agree', 'error', 'swirl', 'argue', 'bleed', 'delta', 'flick', 'totem', 'wooer', 'front', 'shrub', 'parry', 'biome', 'lapel', 'start', 'greet', 'goner', 'golem', 'lusty', 'loopy', 'round', 'audit', 'lying', 'gamma', 'labor', 'islet', 'civic', 'forge', 'corny', 'moult', 'basic', 'salad', 'agate', 'spicy', 'spray', 'essay', 'fjord', 'spend', 'kebab', 'guild', 'aback', 

{'secret_word': 'lunge',
 'word_length': 5,
 'max_guesses': 7,
 'grid': [['', '', '', '', ''],
  ['', '', '', '', ''],
  ['', '', '', '', ''],
  ['', '', '', '', ''],
  ['', '', '', '', ''],
  ['', '', '', '', ''],
  ['', '', '', '', '']],
 'color_grid': [['w', 'w', 'w', 'w', 'w'],
  ['w', 'w', 'w', 'w', 'w'],
  ['w', 'w', 'w', 'w', 'w'],
  ['w', 'w', 'w', 'w', 'w'],
  ['w', 'w', 'w', 'w', 'w'],
  ['w', 'w', 'w', 'w', 'w'],
  ['w', 'w', 'w', 'w', 'w']],
 'i': 0,
 'j': 0,
 'status': 'playing',
 'wordle_words_file': 'wordle_words.txt',
 'all_words_file': 'english_dict.txt',
 'popup_queue': [],
 'active_letters': {'A': True,
  'B': True,
  'C': True,
  'D': True,
  'E': True,
  'F': True,
  'G': True,
  'H': True,
  'I': True,
  'J': True,
  'K': True,
  'L': True,
  'M': True,
  'N': True,
  'O': True,
  'P': True,
  'Q': True,
  'R': True,
  'S': True,
  'T': True,
  'U': True,
  'V': True,
  'W': True,
  'X': True,
  'Y': True,
  'Z': True}}

In [120]:
update_game_history(game_history=game_history, game_state=game_state)
game_history

max_guess_limit : 7
game_hist_len is 1
max_guess_hist : 7
game_history_list : [0, 0, 0, 0, 0, 0, 0]
new_guess_hist_limit : 8
new_list_size : 1
extendable_list : [0]
new_game_history_list : [0, 0, 0, 0, 0, 0, 0, 0]
game_history: [{'game_hist_length': 7, 'game_hist_list': [0, 0, 0, 0, 0, 0, 0], 'old_game_hist_list': []}, {'game_hist_length': 8, 'game_hist_list': [0, 0, 0, 0, 0, 0, 0, 0], 'old_game_hist_list': [0, 0, 0, 0, 0, 0, 0]}]


[{'game_hist_length': 7,
  'game_hist_list': [0, 0, 0, 0, 0, 0, 0],
  'old_game_hist_list': []},
 {'game_hist_length': 8,
  'game_hist_list': [0, 0, 0, 0, 0, 0, 0, 0],
  'old_game_hist_list': [0, 0, 0, 0, 0, 0, 0]}]

In [ ]:
def show_popup(game_state, content, mode="text"):
    if mode =="text":
        game_state['popup_queue'].append((content, mode))
    elif mode == "history":
        print(f"{content[0]} : Number of games lost:  \n")
        for i in range(1, len(content)):
            if content[i] > 0:
                print(f"{content[i]} : Number of games won in {i} Guess(es)\n")
    return

In [166]:
test_game_state = {
    'secret_word':'serve',
    'max_guesses':4,
    'word_length':5,
    'grid':[['s','h','i','n','e'],['s','h','i','u','r']],
    'color_grid':[['g','-','-','-','g'],['g','-','-','-','y']]}

In [171]:
hint_list = []
green_letters = []
green_letters_list = []
green_letter_indices = []
yellow_letters = []
filter_words = []
filtered_hint_list = []
game_grid = test_game_state['grid']
color_grid = test_game_state['color_grid']
word_len = test_game_state['word_length']
file_name = os.path.join(resource_dir,'english_dict.txt')
for i,color_list in enumerate(color_grid):
    for j,color in enumerate(color_list):
        if color =='g':
            green_letters.append({'char':game_grid[i][j],'pos':j})
            green_letters_list.append(game_grid[i][j])
            green_letter_indices.append(j)
        elif color =='y':
            yellow_letters.append(game_grid[i][j])

print(f"green letters: {green_letters}")
filter_words.append(test_game_state['secret_word'])
for word in game_grid:
    guess_word = "".join(word)
    filter_words.append(guess_word)
with open(file_name, 'r') as file:
    all_words = [line.strip() for line in file.readlines() if len(line.strip())==word_len]
filtered_word_list = list(filter(lambda x: x not in filter_words, all_words))
unique_green_list = [dict(t) for t in {frozenset(d.items()) for d in green_letters}]
print(f"unique green list: {unique_green_list}")
for s in filtered_word_list:
    all_match = True  # Flag to track if all characters match
    for correct in unique_green_list:
        char = correct['char']
        index = correct['pos']
        
        # Ensure index is an integer (if it's a string, convert it to integer)
        if isinstance(index, str):
            index = int(index)  # Convert index to integer if it's a string
        
        if len(s) <= index or s[index] != char:  # Check if the char matches the string at the specified index
            all_match = False
            break  # Exit loop early if any condition fails

    if all_match:  # If all characters matched at the correct positions, add the word to the hint_list
        hint_list.append(s)
for s in hint_list:
    # Create a new string by omitting characters at the specified indices
    modified_string = ''.join([s[i] for i in range(len(s)) if i not in green_letter_indices])
    
    # Check if the remaining characters match the condition (contains all chars in char_list)
    if all(char in modified_string for char in yellow_letters):
        filtered_hint_list.append(s)
print(f"hint list : \n {filtered_hint_list}")
    

green letters: [{'char': 's', 'pos': 0}, {'char': 'e', 'pos': 4}, {'char': 's', 'pos': 0}]
unique green list: [{'char': 'e', 'pos': 4}, {'char': 's', 'pos': 0}]
hint list : 
 ['sabre', 'saree', 'sarge', 'scare', 'score', 'scrae', 'scree', 'seare', 'serge', 'serre', 'share', 'shere', 'shire', 'shore', 'siree', 'skyre', 'smore', 'snare', 'snore', 'soare', 'soree', 'spare', 'spire', 'spore', 'spree', 'sprue', 'spyre', 'stare', 'stere', 'stire', 'store', 'strae', 'sture', 'styre', 'sucre', 'surge', 'sware', 'swire', 'swore']


In [139]:
persist_game_stats(game_state=game_state)

In [142]:
def clear_game_state():
    if os.path.exists("game_state.json"):
        os.remove("game_state.json")

In [ ]:
def reset_game(input_game_state, secret_word=None):
    try:
        global game_state
        all_words_file = input_game_state['all_words_file']
        wordle_words_file = input_game_state['wordle_words_file']
        print("test11")
        show_popup(game_state,"Restarting game")
        game_state = initialize_game(all_words_file, wordle_words_file, secret_word)
    except Exception as e:
        print(str(e))



In [ ]:
def validate_guess(game_state):
    valid_guess_ind = False
    try:
        all_words_file = game_state['all_words_file']
        word_len = game_state['word_length']
        file_name = os.path.join(resource_dir,all_words_file)
        if os.path.isfile(file_name):
            with open(file_name, 'r') as file:
                lines = [line.strip() for line in file.readlines() if len(line.strip())==word_len]
                if len(lines)==0:
                    raise Exception("No valid words in all words file")
                else:
                    i = game_state['i']
                    current_guess = "".join(game_state['grid'][i])
                    print(f"Current guess : {current_guess}")
                    if len(current_guess) == word_len and current_guess in lines:
                        valid_guess_ind = True
                    else:
                        show_popup(game_state,"Invalid word\nUse clear or delete to edit your guess")
        else:
            raise Exception("No valid words in all words file")
        
    except Exception as e:
        print(str(e))
    return valid_guess_ind



In [ ]:

def process_guess(guess, secret_word):
    #Assume the guess is list and secret word are str
    color_list = []
    guess_word = "".join(guess)
    #secret_list = ",".split(secret_word)
    matched_list = []
    #print(secret_list)
    # check char in guess word whether it is present , if so,whether it is repeated
    try:
        if guess_word and secret_word:
            if guess_word == secret_word:
                color_list=['g','g','g','g','g']
            else:
                for i in range(5):
                    if guess[i] in secret_word and guess[i] not in matched_list:
                        if guess[i] == secret_word[i]:
                            matched_list.append(secret_word[i])
                            color_list.append('g')
                            #secret_list[i] = '-'
                        else:
                            color_list.append('y')
                    else:
                        color_list.append('-')
                        game_state['active_letters'][guess[i]] = False
            #new_secret = "".join(secret_list)
            #game_state['secret_word']=new_secret
        else:
            print("Either guess or secret_word is Empty")

    except Exception as e:
        print(str(e))
    return color_list


In [ ]:

def process_command(game_state, command):
    process_ind = True
    i = game_state['i']
    print(f"i is {i}")
    j = game_state['j']
    print(f"j is {j}")
    word_len = game_state['word_length']
    special_char_list = re.findall(r"[^a-zA-Z0-9]+", command)
    command_list = ['delete','clear','enter','restart','share','history','quit']
    try:
        #print(f"re.fullmatch('^[A-Z]+$|^[a-z]+$',command) : {re.fullmatch('^[A-Z]+$|^[a-z]+$',command)}")
        if len(command) > 1 and len(special_char_list)>0:
            index = command.find(special_char_list[0])
            word_str = command[:index]
            if word_str in command_list:
                process_command(game_state=game_state,command=word_str)
            else:
                if len(word_str) == word_len:
                    word_str_list = word_str.split(",")
                    for command in word_str_list:
                        process_command(game_state=game_state,command=command)
            process_command(game_state=game_state,command=special_char_list[0])
        elif len(command)==1 and re.fullmatch('^[A-Z]+$|^[a-z]+$',command):
            # check the length of current guess
            print("test1")
            if j < word_len:
                print("test2")
                game_state['grid'][i][j] = command
                print(f"{game_state['grid'][i][j]}")
                game_state['j'] = j+1
            else:
                message="Can't add more letters\nUse clear or delete to edit your guess."
                show_popup(game_state,message)
        elif command == 'delete' or command == '-':
            print("test3")
            game_state['grid'][i][j-1] = ''
            j-=1
            game_state['j'] = j
        elif command == 'clear' or command == '_':
            print("test4")
            game_state['grid'][i]=['','','','','']
            j = 0
            game_state['j'] = j
            print(f"{game_state['j']}")
        elif command == 'enter' or command == '+':
            print("test5")
            if validate_guess(game_state):
                print("test6")
                curr_guess = "".join(game_state['grid'][i])
                color_list = process_guess(curr_guess,game_state['secret_word'])
                print(f"color_list is {color_list}")
                game_state['colour_grid'][i]=color_list
                game_state['max_guesses'] -=1
                if len(set(game_state['colour_grid'][i])) == 1 and set(game_state['colour_grid'][i])=='g':
                    print("test7")
                    game_state['status']='won'
                    update_game_history(game_history,game_state)
                    clear_game_state()
                    show_popup(game_state,"You won!")
                    process_ind = False
                else:
                    print("test8")
                    if game_state['max_guesses']==0 and game_state['i']==word_len:
                        print("test9")
                        game_state['status']='lost'
                        update_game_history(game_history,game_state)
                        clear_game_state()
                        message = f"You lose! The word was {game_state['secret_word']}"
                        show_popup(game_state,message)
                        process_ind = False
                    else:
                        print("test10")
                        game_state['i'] += 1
                        game_state['j'] = 0
        elif command == 'restart' or command == '!':
            print("test")
            message=f"You have chosen to restart! The word was {game_state['secret_word']}"
            show_popup(game_state,message)           
            reset_game(game_state)
        elif command == 'share' or command =='@':
            game_status = game_state['status']
            game_result_str = ""
            if game_status == 'won' or game_status == 'lost':
                guess_result = ""
                results_list = game_state['color_grid']
                for guess_result in results_list:
                    for char_result in guess_result:
                        if char_result == 'g':
                            unicode_char = "\U0001F7E9"
                        elif char_result == 'y':
                            unicode_char = "\U0001F7E8"
                        elif char_result == '-':
                            unicode_char = "\U00002B1C"
                        guess_result=str(guess_result)+ str(unicode_char)
                    game_result_str=str(game_result_str)+str(guess_result+"\n")  
                show_popup(game_state=game_state,content=game_result_str,mode="share")
        elif command == 'history':
            if os.path.exists("game_history.txt"):
                with open("game_history.txt", "r") as file:
                    data = file.read().strip().split(",")
                    history = [int(x) for x in data]  
                    show_popup(game_state=game_state,content=history,mode="history")
        elif command == 'quit':
            process_ind = False
            message = "You have chosen to quit! Game progress will be saved!"
            show_popup(game_state=game_state,content=history,mode="text")
            sys.exit()    
        else:
            show_popup(game_state,"Invalid command")
        print(game_state)
    except KeyboardInterrupt as err:
        persist_game_stats(game_state)
        print(str(err))
    except SystemExit as sysexit:
        persist_game_stats(game_state)
        print(str(sysexit))
    except Exception as e:
        persist_game_stats(game_state)
        print(str(e))
    return process_ind


In [34]:
print("\U0001F7E9\U00002B1C\U0001F7E8")
print("\U00002B1C")
print("\U0001F7E8")
print("\U00002B1B")

🟩⬜🟨
⬜
🟨
⬛


In [238]:
print(game_state)
#game_state['j']=5

{'secret_word': 'sneer', 'word_length': 5, 'max_guesses': 0, 'grid': [['c', 'h', 'e', 'e', 'r'], ['c', 'h', 'o', 's', 'e'], ['c', 'h', 'o', 'o', 'k'], ['f', 'l', 'a', 'n', 'k'], ['f', 'l', 'a', 'w', 'n'], ['f', 'l', 'e', 'e', 'k']], 'colour_grid': [['-', '-', 'g', '-', 'g'], ['-', '-', '-', 'y', 'y'], ['-', '-', '-', '-', '-'], ['-', '-', '-', 'y', '-'], ['-', '-', '-', '-', 'y'], ['-', '-', 'g', '-', '-']], 'i': 5, 'j': 5, 'status': 'lost', 'wordle_words_file': 'wordle_words.txt', 'all_words_file': 'english_dict.txt', 'popup_queue': [], 'active_letters': {}}


In [230]:
game_state['status']='playing'

In [228]:
game_state['colour_grid'][5]=['w','w','w','w','w']

In [226]:

game_state['max_guesses']=1

In [231]:
game_state['i']=5
game_state['j']=5

In [259]:
process_command(game_state=game_state,command='restart')


i is 0
j is 0
re.fullmatch('^[A-Z]+$|^[a-z]+$',command) : <regex.Match object; span=(0, 7), match='restart'>
test
test11
{'secret_word': 'boule', 'word_length': 5, 'max_guesses': 6, 'grid': [['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', '']], 'colour_grid': [['w', 'w', 'w', 'w', 'w'], ['w', 'w', 'w', 'w', 'w'], ['w', 'w', 'w', 'w', 'w'], ['w', 'w', 'w', 'w', 'w'], ['w', 'w', 'w', 'w', 'w'], ['w', 'w', 'w', 'w', 'w']], 'i': 0, 'j': 0, 'status': 'playing', 'wordle_words_file': 'wordle_words.txt', 'all_words_file': 'english_dict.txt', 'popup_queue': [], 'active_letters': {}}


True